In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available



In [3]:

if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [4]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation 4 500GB Console (Renewed),,4.4 out of 5 stars,"1,163 ratings",In Stock
1,PlayStation 4 Slim 1TB Console - Black (Renewed),,4.5 out of 5 stars,"1,603 ratings",In Stock
2,"Sony PlayStation 4 Pro w/ Accessories, 1TB HDD...",,4.5 out of 5 stars,943 ratings,In Stock
5,"Playstation SONY 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,431 ratings,Only 9 left in stock - order soon.
6,PlayStation 4 Slim 1TB Console,,4.5 out of 5 stars,"15,706 ratings",In Stock
8,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,345 ratings,Only 3 left in stock - order soon.
12,PlayStation®5 Digital Edition (slim),,4.7 out of 5 stars,"5,793 ratings",Not Available
15,RPlay Play-Station 4 PS4 1TB Slim Edition Jet ...,,4.5 out of 5 stars,57 ratings,In Stock
18,PlayStation 4 500GB Console (Renewed),,4.4 out of 5 stars,"1,163 ratings",In Stock
23,PlayStation 3 Slim Console 120GB (Old Model) (...,,4.4 out of 5 stars,263 ratings,Only 1 left in stock - order soon.
